In [1]:
#libraries
import thesisFunctions
import pandas as pd
import numpy as np
import os
import sys
#import simpledbf
import math
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/home/pipe/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#download data from INDEC
#thesisFunctions.getEPHInd(trimestre = 't310', path = os.getenv('TESISDATA'))


In [2]:
import pysal as ps
def caca(dbfile, upper=True): #Reads in DBF files and returns Pandas DF
    '''
    Arguments
    ---------
    dbfile  : DBF file - Input to be imported
    upper   : Condition - If true, make column heads upper case
    '''
    db = ps.open(dbfile) #Pysal to open DBF
    d = {col: db.by_col(col) for col in db.header} #Convert dbf to dictionary
    pandasDF = pd.DataFrame(db[:]) #Convert to Pandas DF
    pandasDF.columns=db.header
    pandasDF.dropna(inplace=True)
    return pandasDF

In [ ]:
dbfile = '../data/Individual_t310.dbf'
db = ps.open(dbfile)
db.header
#d = {col: db.by_col(col) for col in db.header} #Convert dbf to dictionary
for col in db.header:
    print db.by_col(col)

In [4]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [5]:
#read data
#rawInd = thesisFunctions.readEPHInd(trimestre = 't310', path = os.getenv('TESISDATA'))
data = caca('../data/Individual_t310.dbf')

UnicodeDecodeError: 'utf8' codec can't decode byte 0xa0 in position 22: invalid start byte

In [ ]:
data.head()

In [ ]:
#clean data
cleanInd = thesisFunctions.cleanEPHInd(rawInd)
#dump to csv
cleanInd.to_csv(os.getenv('TESISDATA') + '/cleanInd.csv', index=False)

In [ ]:
data1 = pd.read_csv(os.getenv('TESISDATA') + '/cleanInd.csv')
data1.head()

In [ ]:
data2 = thesisFunctions.categorizeInd(data1)
data3 = thesisFunctions.schoolYears(data2)
data4 = thesisFunctions.make_dummyInd(data3)
data5 = thesisFunctions.createVariablesInd(data4)

In [ ]:
sns.kdeplot(data5.lnIncome)

In [ ]:
jobsAndIncome = ((data5.activity==1) & (data5.lnIncome>1))
data6 = data5.copy().loc[jobsAndIncome,:]

In [ ]:
sns.kdeplot(data6.lnIncome)

In [ ]:
print data6.shape

In [ ]:
out = ((data6.lnIncome < 4) | (data6.lnIncome > 10.))
data6 = data6.copy().loc[~out,:]

In [ ]:
print data6.shape

In [ ]:
g = sns.JointGrid(x="education", y="lnIncome", data=data6)  
g.plot_joint(sns.regplot, order=2)  
g.plot_marginals(sns.distplot)

g2 = sns.JointGrid(x="age", y="lnIncome", data=data6)  
g2.plot_joint(sns.regplot, order=2)  
g2.plot_marginals(sns.distplot)

In [ ]:
model = thesisFunctions.runModel(data6, income = 'lnIncome', variables= [
        'primary','secondary','university',
        'male_14to24','male_25to34',
        'female_14to24', 'female_25to34', 'female_35more'])

In [ ]:
variables= [
        'primary','secondary','university',
        'male_14to24','male_25to34',
        'female_14to24', 'female_25to34', 'female_35more']
    
X = sm.add_constant(data6.copy().loc[:,variables].values)
data6['predicted'] = model.predict(X)

In [ ]:
#plt.plot(np.linspace(4,11,100),np.linspace(4,11,100),'b-')
plt.plot(data6.lnIncome,data6.predicted,'ro')


In [ ]:
data6['resid'] = data6.lnIncome - data6.predicted 
print data6.resid.mean()

In [ ]:
plt.plot(data6.predicted,data6.resid,'ro')

## Aplicar nuevos coeficientes

## Reproducir capeceo en EPH

## Correr modelo de regresion simple con CAPECO

## Computar otras variables del modelo factorial de Mariana

## Correr Lasso y Ridge con todas estas variables